In [1]:
import matplotlib.pyplot as plt

from obspy.clients.fdsn.client import Client
from obspy import UTCDateTime
from obspy.core.event import Catalog
from obspy.core.stream import Stream
from obspy.core.event import Event
from obspy.core.event import Origin
from obspy.core.event import Magnitude
from obspy import read_events
from obspy import read_inventory
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import pickle


/Users/user/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
catINGV=read_events('flegrei_2014_2023_INGV/catalogue_flegrei.xml')

In [2]:
catOV=read_events('flegrei_2014_2023_GOSSIP/catalogue_flegrei.xml')

In [ ]:
str(catOV.events[0].resource_id).split('/')[1]

In [ ]:
id_catalogues=[]

for ind,ev in enumerate(catINGV):
    
    origin_time = ev.origins[0].time
    time_min= 'time > ' + str(origin_time - 5)
    time_max= 'time < ' + str(origin_time + 5)
    cat= catOV.filter(time_min , time_max)

    id_catalogues.append([])
    id_catalogues[ind].append( int( str(ev.resource_id).split('=')[1] ) )

    for match in cat:
        id_catalogues[ind].append( int( str(match.resource_id).split('/')[1] ) )

id_catalogues

In [ ]:
print( 'len INGV:', len(catINGV), 'len comparison:', len(id_catalogues) )

## save

In [ ]:
with open("event_comparison_INGV_GOSSIP", "wb") as fp:   #Pickling
    pickle.dump(id_catalogues, fp)

## load

In [ ]:
with open("event_comparison_INGV_GOSSIP", "rb") as fp:   # Unpickling
    catalogues = pickle.load(fp)

In [ ]:
catalogues[0:6]

In [ ]:
ev_not_found=[]
ev_multiple_det=[]

for ev in catalogues:
    if len(ev)   >= 3:
        ev_multiple_det.append(ev)
    elif len(ev) == 1:
        ev_not_found.append(ev)
    elif len(ev) != 2:
        print('!event not categorized!')
        

In [ ]:
ev_multiple_det

In [ ]:
ev_not_found

In [ ]:
[ev for ev in catOV if ev.resource_id.id.split('/')[1]== str(catalogues[0][1]) ]

In [ ]:
for ev in catOV:
    if  ev.resource_id.id.split('/')[1]== str(catalogues[0][1]):
        ev1=ev
ev1

# Stations

In [3]:
client=Client('INGV')

stime=UTCDateTime('2014-01-01T00:00:00')
etime=UTCDateTime('2023-11-20T00:00:00')

In [ ]:
lat=40.8478
long=14.0918
rad_events=0.5
inv=client.get_stations(starttime=stime,endtime=etime,
                        latitude=lat,longitude=long,maxradius=rad_events,
                        level='response')                                   #instrumental response
inv

In [ ]:
inv.plot(projection='local',resolution='h');

In [ ]:
inv.write('flegrei_2014_2023_INGV/inventory_flegrei.xml',format='STATIONXML')                        #save

In [4]:
inv_f=read_inventory('flegrei_2014_2023_INGV/inventory_flegrei.xml')                                 #read

In [ ]:
inv_f.plot_response(min_freq=1E-4,station='SORR',channel='[EH]H?');

# Waveform 

### from Magnitude to origin time


In [ ]:
catOV_mag = catOV.filter("magnitude >= 3.0")
for ev in catOV_mag:
    origin_time=ev.origins[0].time
    print('origin time event:',origin_time)
    print('extimated magnitude:',ev.magnitudes[0].mag)

### from ID to origin time

In [ ]:
evID = str(ev_multiple_det[0][2])

for ev in catOV :
    if ev.resource_id.id.split('/')[1]== evID:
        origin_time=ev.origins[0].time
        print('event occurred:',origin_time)
        print('extimated magnitude:',ev.magnitudes[0].mag)


In [ ]:
inv_f

In [5]:
catOV_mag = catOV.filter("magnitude >= 3.0")
for ev in catOV_mag:
    origin_time=ev.origins[0].time
    evID=ev.resource_id.id.split('/')[1]
    print('event id: ',evID)
    print('origin time event:',origin_time)
    print('extimated magnitude:',ev.magnitudes[0].mag)


    event_start=origin_time -10
    event_end=origin_time +50

    wave=Stream()
    for network in inv_f:
        for  station in network.stations:
            try:
                wave += client.get_waveforms(starttime=event_start,endtime=event_end,
                                    network=network.code,station=station.code,location='*', channel='HH?',
                                    attach_response=True,)
            except:
                print(station.code , 'station not found')
                #continue

    print('traces found:',len(wave.traces))

    wave.merge(fill_value=0)
    # trim over the [t1, t2] interval
    wave.trim(starttime=event_start, endtime=event_end, pad=True, fill_value=0)

    '''
    # remove trend
    wave.detrend("demean")
    
    #remove instrumental response
    pre_filt = [0.01, 0.1, 25,30]
    
    #remove instrumental response
    wave.remove_response(inventory=inv_f, output='DISP', pre_filt=pre_filt) #output=VEL ??
    '''

    wave.write('waveform/wave_'+ evID +'.mseed',format='MSEED')
    print('saved!')

event id:  2508
origin time event: 2019-12-06T00:17:23.780000Z
extimated magnitude: 3.1
CAFL station not found
CAWE station not found
CBAC station not found
CCAP station not found
CMTS station not found
CNIS station not found
CQUE station not found
CRTO station not found
CSTH station not found
IBCM station not found
ICVJ station not found
IPSM station not found
IVLC station not found
OVO station not found
PTMR station not found
SORR station not found
VARP station not found
VBKE station not found
VBKS station not found
VBKW station not found
VCNE station not found
VPOB station not found
VTRZ station not found
NAPI station not found
T1361 station not found
T1363 station not found
T1365 station not found
T1366 station not found
T1367 station not found
T1368 station not found
traces found: 57
saved!
event id:  7102
origin time event: 2022-03-16T14:14:34.800000Z
extimated magnitude: 3.47


/Users/user/Library/Python/3.9/lib/python/site-packages/obspy/io/mseed/core.py:821: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')


CAFL station not found
CAWE station not found
CCAP station not found
CMTS station not found
CNIS station not found
CPIS station not found
CQUE station not found
IBCM station not found
ICVJ station not found
IPSM station not found
IVLC station not found
MODR station not found
PAOL station not found
PTMR station not found
VARP station not found
VBKS station not found
VBKW station not found
VCNE station not found
VCRE station not found
VPOB station not found
VTRZ station not found
NAPI station not found
T1361 station not found
T1363 station not found
T1365 station not found
T1366 station not found
T1367 station not found
T1368 station not found
traces found: 63


/Users/user/Library/Python/3.9/lib/python/site-packages/obspy/io/mseed/core.py:821: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')


saved!
event id:  14725
origin time event: 2022-03-29T17:45:32.360000Z
extimated magnitude: 3.57
CAFL station not found
CAWE station not found
CCAP station not found
CMTS station not found
CPIS station not found
CQUE station not found
IBCM station not found
ICVJ station not found
